In [1]:
import sys
import re
import math
from collections import Counter
!pip install pyspark
from pyspark import SparkContext, SparkConf
from random import randint
import shutil
# shutil.rmtree('')
# !unzip TF_index.zip
conf = SparkConf()
conf.setMaster('local')
conf.setAppName('TF/IDF')
sc = SparkContext.getOrCreate(conf=conf)

     |████████████████████████████████| 212.3MB 71kB/s 
     |████████████████████████████████| 204kB 18.9MB/s 
  Created wheel for pyspark: filename=pyspark-3.1.1-py2.py3-none-any.whl size=212767604 sha256=010a9871487f38f80a4ba1f7c600a6cb5c7841a00e10df91cab73a976c57369c
  Stored in directory: /root/.cache/pip/wheels/0b/90/c0/01de724414ef122bd05f056541fb6a0ecf47c7ca655f8b3c0f
Successfully built pyspark


In [15]:
# Read CTF_index
# a. Retrieve cosine normalized vector of the query words from CTF_index
def st_to_dict(line):
  line = line.split("@")
  line1 = line[1].split("+")
  return (line[0], line[1])


# read data
# Edit file name to give your input data
dataFile = 'CTF_index'
# create RDDS, read stopwords file
stopWordFile ='stopwords-en.txt'
tf_RDD = sc.textFile(dataFile)
tf_RDD1 = tf_RDD.map(st_to_dict)
# tf_RDD1.take(10)

# Calculate N (document size)
countRDD2=tf_RDD1.flatMap(lambda x: x[1].split('+'))
countRDD2=countRDD2.map(lambda x: (x.split('#')[0],x.split('#')[1]))
countRDD = countRDD2.map(lambda x: x).keys().distinct()
N = countRDD.count()

In [10]:
############## Pre process

# Remove punctutaion function

def lower_clean_str(x):
  punc='!"#$%&\'()*+,;=?@[]^_`{|}~-'
  # punc='!"#$%&\'()*+,./:;<=>?@[\\]^_`{|}~-'
  lowercased_str = x.lower()
  for ch in punc:
    lowercased_str = lowercased_str.replace(ch, ' ')
    lowercased_str = ' '.join(lowercased_str.split())
  return lowercased_str

def lower_clean_str2(x):
  # punc='!"#$%&\'()*+,;=?@[]^_`{|}~-'
  punc='!"#$%&\'()*+,./:;<=>?@[\\]^_`{|}~-'
  lowercased_str = x.lower()
  for ch in punc:
    lowercased_str = lowercased_str.replace(ch, ' ')
    lowercased_str = ' '.join(lowercased_str.split())
  return lowercased_str


def remove_urls_tags(x):
  regex = r"(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'\".,<>?«»“”‘’]))"
  url_str = re.sub(regex, " ", x)
  html_regex = r"<.*?>"
  finalStr = re.sub(html_regex, " ", url_str)
  return finalStr

# Remove stop words function
def remove_stop_words(stri):
  with open(stopWordFile,'r') as stop:
    stop_w = stop.read()
    #splitting stopwords.txt by new line
    stop_list = re.split('\n', stop_w)
    for word in stop_list:
        if word in stri:
            stri = stri.replace(" "+ word + " ", " ")
    return stri

# Preprocess function
def pre_process(fileRDD):
  #Run remove punctuation function on data
  fileRDD = fileRDD.map(lower_clean_str)
  # Remove url_tags
  fileRDD = fileRDD.map(remove_urls_tags)
  fileRDD = fileRDD.map(lower_clean_str2)
  #Remove stop words
  fileRDD1 = fileRDD.map(remove_stop_words)
  fileRDD1 = fileRDD1.map(lambda line : line.strip())
  return fileRDD1

In [12]:
qids=set()
# boolean variable for while check
if_query=True

#while incoming queries
while(if_query):
  #user submits a query
  qry=input("Enter Query")
  a = True
  # Generate randome qID
  qID=0
  while(a):
     qID=0
     _id=randint(100,999)
     if _id not in qids:
        qids.add(_id)
        a=False
        qID=_id
  #convert query to RDD
  rdd = sc.parallelize([qry])
  # all text pre-processing, given in Step-1
  prdqry=pre_process(rdd)
  words = prdqry.collect()[0].split(" ")
  prdFilter=tf_RDD1.filter(lambda x: str(x[0]) in words)
  prdFilter.take(5)
  # ///////////////// Calculate IDF
  prd3 = prdFilter.map(lambda x : (x[0], len(x[1].split("+"))))
  prd3.take(5)
  prdIDF=prd3.map(lambda x: (x[0],math.log10(N/(1+x[1]))))
  prdIDF.take(25)

  # b. Construct TF-IDF (Ct) for each word in the query (NOTE: TF of a word in the query may NOT be always 1). You can calculate IDF and TF-IDF values from CTF_index
  #  Calculate TF
  tf1=prdFilter.map(lambda x:(x[0], x[1]))
  tf2=tf1.map(lambda x:(x[0],x[1].split('+')))
  tf2.take(10)
  #  Function to break tuple
  def break_tup(tp):
    res = (tp[0],[(i.split("#")[0], 1+math.log10(float(i.split("#")[1]))) for i in tp[1]])
    return res
  # Now transform the RDD so that it will be of format: (word, (doc. ID, freq))
  tfFinal = tf2.map(break_tup)
  tfFinal.take(5)

  # TF.IDF Dot product we take join 
  tfidfJoin = prdIDF.join(tfFinal)
  tfidfJoin.take(5)

  # dot product (tf*idf)
  # c. Perform DOT product on two vectors (Ct and CTFd)
  def extract_join(jd):
      wrd = jd[0]
      idf = jd[1][0]
      lis = jd[1][1]
      li =[]
      for x in lis:
        li.append((x[0], ((float)(x[1]) *(float)(idf))))
      return li

  tfExtr = tfidfJoin.flatMap(extract_join)
  tfExtr.take(25)

  # sum each vector
  # d. Sort the documents in decreasing order of the score and write only top 10 Doc. IDs to qID directory
  countRDD = tfExtr.reduceByKey(lambda a,b: a+b).sortBy(lambda a: a[1],ascending=False).map(lambda x : x[0]).take(10)
  writeRDD =  sc.parallelize(countRDD)
  writeRDD.saveAsTextFile(str(qID))
  #  Continue queries ?
  nextQR=input('Continue ? Enter Y or N')
  if_query = True if (nextQR=="Y" or nextQR=="y") else False

Enter QueryParis india-1947
Continue ? Enter Y or Nn
